In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e16/sample_submission.csv
/kaggle/input/playground-series-s3e16/train.csv
/kaggle/input/playground-series-s3e16/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s3e16/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s3e16/test.csv')

In [3]:
train.head()

,id,Sex,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
0,0,I,1.5250,1.1750,0.3750,28.973189,12.728926,6.647958,8.348928,9
1,1,I,1.1000,0.8250,0.2750,10.418441,4.521745,2.324659,3.401940,8
2,2,M,1.3875,1.1125,0.3750,24.777463,11.339800,5.556502,6.662133,9
3,3,F,1.7000,1.4125,0.5000,50.660556,20.354941,10.991839,14.996885,11
4,4,I,1.2500,1.0125,0.3375,23.289114,11.977664,4.507570,5.953395,8


In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74051 entries, 0 to 74050
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              74051 non-null  int64  
 1   Sex             74051 non-null  object 
 2   Length          74051 non-null  float64
 3   Diameter        74051 non-null  float64
 4   Height          74051 non-null  float64
 5   Weight          74051 non-null  float64
 6   Shucked Weight  74051 non-null  float64
 7   Viscera Weight  74051 non-null  float64
 8   Shell Weight    74051 non-null  float64
 9   Age             74051 non-null  int64  
dtypes: float64(7), int64(2), object(1)
memory usage: 5.6+ MB


In [5]:
train['Sex'].value_counts()

M    27084
I    23957
F    23010
Name: Sex, dtype: int64

In [6]:
train.describe()

,id,Length,Diameter,Height,Weight,Shucked Weight,Viscera Weight,Shell Weight,Age
count,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000,74051.000000
mean,37025.000000,1.317460,1.024496,0.348089,23.385217,10.104270,5.058386,6.723870,9.967806
std,21376.826729,0.287757,0.237396,0.092034,12.648153,5.618025,2.792729,3.584372,3.175189
min,0.000000,0.187500,0.137500,0.000000,0.056699,0.028349,0.042524,0.042524,1.000000
25%,18512.500000,1.150000,0.887500,0.300000,13.437663,5.712424,2.863300,3.968930,8.000000
50%,37025.000000,1.375000,1.075000,0.362500,23.799405,9.908150,4.989512,6.931453,10.000000
75%,55537.500000,1.537500,1.200000,0.412500,32.162508,14.033003,6.988152,9.071840,11.000000
max,74050.000000,2.012815,1.612500,2.825000,80.101512,42.184056,21.545620,28.491248,29.000000


In [7]:
corr_matrix = train.corr()

/tmp/ipykernel_20/1550658072.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = train.corr()


In [8]:
corr_matrix['Age'].sort_values(ascending=False)

Age               1.000000
Shell Weight      0.663473
Height            0.638067
Diameter          0.621256
Length            0.612843
Weight            0.601195
Viscera Weight    0.576808
Shucked Weight    0.503320
id                0.000089
Name: Age, dtype: float64

In [9]:
#Adding Feature engineering

train['viscera_ratio'] = train['Viscera Weight'] / train['Weight']
train['shell_ratio'] = train['Shell Weight'] / train['Weight']
train['Surface Area'] = 2 * (train['Length'] * train['Diameter'] + train['Length'] * train['Height'] + train['Diameter'] * train['Height'])
train['Volume'] = train['Length'] * train['Diameter'] * train['Height']

test['viscera_ratio'] = test['Viscera Weight'] / test['Weight']
test['shell_ratio'] = test['Shell Weight'] / test['Weight']
test['Surface Area'] = 2 * (test['Length'] * test['Diameter'] + test['Length'] * test['Height'] + test['Diameter'] * test['Height'])
test['Volume'] = test['Length'] * test['Diameter'] * test['Height']

In [10]:
corr_matrix = train.corr()

/tmp/ipykernel_20/1550658072.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  corr_matrix = train.corr()


In [11]:
corr_matrix['Age'].sort_values(ascending=False)

Age               1.000000
Shell Weight      0.663473
Height            0.638067
Surface Area      0.621660
Diameter          0.621256
Length            0.612843
Volume            0.607129
Weight            0.601195
Viscera Weight    0.576808
Shucked Weight    0.503320
shell_ratio       0.091076
id                0.000089
viscera_ratio    -0.046799
Name: Age, dtype: float64

In [12]:
X = train.drop('Age', axis=1)
y = train['Age'].copy()

In [13]:
categorical_cols = X.select_dtypes(include="object").columns
numerical_cols = X.select_dtypes(exclude="object").columns

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

numerical_transformer = Pipeline([('std_scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))
])


preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [15]:
train_prepared = preprocessor.fit_transform(X)
test_prepared = preprocessor.transform(test)

In [16]:
from sklearn.ensemble import RandomForestRegressor


forest = RandomForestRegressor()

In [17]:
forest.fit(train_prepared,y)

RandomForestRegressor()

In [18]:
predictions = forest.predict(train_prepared)

In [19]:
from sklearn.metrics import mean_squared_error 

mse = mean_squared_error(y, predictions)
rmse = np.sqrt(mse)
rmse

0.7865163294059415

In [20]:
mse

0.6186079364221955

In [21]:
from sklearn.model_selection import RandomizedSearchCV

In [22]:
param_distributions = [{'n_estimators':[3,10,30],'max_features':[2,4,6,8]},
                      {'bootstrap':[False],'n_estimators':[3,10],'max_features':[2,3,4]}]
for_search_cv = RandomizedSearchCV(forest, param_distributions,scoring='neg_mean_squared_error', return_train_score=True)
for_search_cv.fit(train_prepared, y)

RandomizedSearchCV(estimator=RandomForestRegressor(),
                   param_distributions=[{'max_features': [2, 4, 6, 8],
                                         'n_estimators': [3, 10, 30]},
                                        {'bootstrap': [False],
                                         'max_features': [2, 3, 4],
                                         'n_estimators': [3, 10]}],
                   return_train_score=True, scoring='neg_mean_squared_error')

In [23]:
final_model = for_search_cv.best_estimator_

In [24]:
final_model

RandomForestRegressor(max_features=2, n_estimators=30)

In [25]:
final_predictions = final_model.predict(test_prepared)

In [26]:
submission_df = pd.DataFrame({'id': test['id'], 'Age': final_predictions})
submission_df.to_csv('submission.csv', index=False)